In [ ]:
ENV_NAME = "anomaly"
ENV_DESCRIPTION = "Docker image for anomaly detection with anomalib."
ENV_DOCKER_BUILDCONTEXT = "docker"
ENV_DOCKERFILE = "Dockerfile"
ENV_VERSION = "4"

DATA_NAME = "mvtec"
DATA_VERSION = "1"

COMPUTE_NAME = "cpu-cluster"

COMPONENT_VERSION = "1"


## Create AzureML Environment - Docker

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, BuildContext
from azure.identity import InteractiveBrowserCredential

# get credentials by logging in to Azure Portal (a new browser tab will open)
credential = InteractiveBrowserCredential()
# establish a client connection to AzureML Studio Workspace
client = MLClient.from_config(credential)
# define docker build
docker = BuildContext(
    path=ENV_DOCKER_BUILDCONTEXT,
    dockerfile_path=ENV_DOCKERFILE,
)
# define environment locally
env = Environment(
    build=docker,
    name=ENV_NAME,
    version=ENV_VERSION,
    description=ENV_DESCRIPTION,
)
# create environment in workspace
client.environments.create_or_update(env)

## Run AzureML Job - Command

In [1]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output
from datetime import datetime
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

timestamp = datetime.now().strftime('%m-%d-%yT%H-%M-%S')

credential = InteractiveBrowserCredential()
client = MLClient.from_config(credential)

env = client.environments.get(ENV_NAME, version=ENV_VERSION)
data = client.data.get(DATA_NAME, version=DATA_VERSION)

job = command(
    experiment_name="carpet-anomaly",
    name=f"train_anomaly_{timestamp}",
    display_name=f"Train Anomaly Model",
    inputs=dict(
        data_dir=Input(path=data.path, type="uri_folder"),
    ),
    outputs=dict(
        output_dir=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code="./src",
    command="python command.py --data_dir ${{inputs.data_dir}} --output_dir ${{outputs.output_dir}}",
    environment=env,
    compute=COMPUTE_NAME,
)
# submit the command
returned_job = client.create_or_update(job)
returned_job

ValidationException: We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.

## Run AzureML Job - Component

In [ ]:

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

ml_env = ml_client.environments.get("anomaly", version="2")
ml_env

In [ ]:
ml_env

In [1]:
from azure.ai.ml import load_component
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential
from src.designer import train_component

train_component = load_component(source="./components/train.yaml")

credential = InteractiveBrowserCredential()
client = MLClient.from_config(credential)
client.components.create_or_update(train_component)

Found the config file in: ./.azureml/config.json
Uploading train (0.01 MBs): 100%|██████████| 9850/9850 [00:00<00:00, 52800.28it/s]




CommandComponent({'auto_increment_version': False, 'source': 'REMOTE.WORKSPACE.COMPONENT', 'is_anonymous': False, 'name': 'train_anomalib', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/32dff789-bcd1-41d0-8899-b06bf1c4a8c7/resourceGroups/cv-framework-rg/providers/Microsoft.MachineLearningServices/workspaces/cv-framework/components/train_anomalib/versions/0.0.10', 'Resource__source_path': None, 'base_path': PosixPath('.'), 'creation_context': <azure.ai.ml._restclient.v2022_10_01.models._models_py3.SystemData object at 0x114f6b400>, 'serialize': <msrest.serialization.Serializer object at 0x114e54bb0>, 'command': 'python command.py --data_dir ${{inputs.data_dir}} $[[--max_epochs ${{inputs.max_epochs}}]] --output_dir ${{outputs.output_dir}}', 'code': '/subscriptions/32dff789-bcd1-41d0-8899-b06bf1c4a8c7/resourceGroups/cv-framework-rg/providers/Microsoft.MachineLearningServices/workspaces/cv-framework/codes/6c91406b-e564-429c-8c54-1e8e8251410

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential
from src.designer import train_component

train_component.dump("tmp.yaml")

# credential = InteractiveBrowserCredential()
# client = MLClient.from_config(credential)
# client.components.create_or_update(train_component)

In [ ]:
# if not exists, register component using following code
prep = client.components.create_or_update(anomalib_train_component)

In [ ]:
from azure.ai.ml.dsl import pipeline

# define a pipeline containing 3 nodes: Prepare data node, train node, and score node
@pipeline(
    default_compute=COMPUTE_NAME,
)
def pipe(data_dir):
    train_component = client.components.get(name="train_anomaly", version="1")

    train_node = train_component(
        data_dir=data_dir
    )


# create a pipeline
pipeline_job = pipe(data_dir)

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job